In [1]:
import pandas as pd

# Load the CSV you uploaded
df = pd.read_csv("financial_data.csv")
df.head()


,Company,Fiscal Year,Total Revenue (USD B),Net Income (USD B),Total Assets (USD B),Total Liabilities (USD B),Cash Flow from Operating Activities (USD B)
0,Microsoft,2023,211.92,72.36,411.98,205.75,87.58
1,Microsoft,2022,198.27,72.74,364.84,198.30,89.04
2,Microsoft,2021,168.09,61.27,333.78,191.79,76.74
3,Apple,2023,383.29,97.00,352.58,290.44,110.54
4,Apple,2022,394.33,99.80,352.76,302.08,122.15


In [2]:
def build_canned_answers(df):
    answers = {}
    for company, group in df.groupby("Company"):
        group = group.sort_values("Fiscal Year", ascending=True)
        latest = group.iloc[-1]
        prev = group.iloc[-2] if len(group) > 1 else None

        answers[company.lower()] = {
            "latest_year": int(latest["Fiscal Year"]),
            "revenue": latest["Total Revenue (USD B)"],
            "net_income": latest["Net Income (USD B)"],
            "cash_flow": latest["Cash Flow from Operating Activities (USD B)"],
            "prev_net_income": prev["Net Income (USD B)"] if prev is not None else None
        }
    return answers

canned = build_canned_answers(df)
canned


{'apple': {'latest_year': 2023,
  'revenue': np.float64(383.29),
  'net_income': np.float64(97.0),
  'cash_flow': np.float64(110.54),
  'prev_net_income': np.float64(99.8)},
 'microsoft': {'latest_year': 2023,
  'revenue': np.float64(211.92),
  'net_income': np.float64(72.36),
  'cash_flow': np.float64(87.58),
  'prev_net_income': np.float64(72.74)},
 'tesla': {'latest_year': 2023,
  'revenue': np.float64(96.77),
  'net_income': np.float64(15.0),
  'cash_flow': np.float64(13.34),
  'prev_net_income': np.float64(12.56)}}

In [3]:
!pip install fuzzywuzzy python-Levenshtein


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from fuzzywuzzy import process

supported_questions = [
    "what is the total revenue for {company}?",
    "how has net income changed over the last year for {company}?",
    "which company had the highest recent revenue growth?",
    "what is the cash flow from operating activities for {company}?"
]

def answer_query(query):
    q = query.strip().lower()
    companies = list(canned.keys())

    # Detect company name
    match_company = None
    for comp in companies:
        if comp in q:
            match_company = comp
            break
    if not match_company:
        result = process.extractOne(q, companies)
        if result and result[1] >= 70:
            match_company = result[0]

    if "total revenue" in q:
        if not match_company:
            return "Please specify a company."
        info = canned[match_company]
        return f"{match_company.title()} total revenue ({info['latest_year']}): {info['revenue']} USD B."
    
    if "net income" in q and "last" in q:
        if not match_company:
            return "Please specify a company."
        info = canned[match_company]
        prev = info['prev_net_income']
        if prev is not None:
            diff = info['net_income'] - prev
            pct = (diff / prev) * 100
            sign = "+" if diff >= 0 else ""
            return f"{match_company.title()} net income: {prev} -> {info['net_income']}. Change: {sign}{diff:.2f} USD B ({pct:.2f}%)."
        else:
            return f"{match_company.title()} net income (latest): {info['net_income']} USD B."
    
    if "highest" in q and "revenue growth" in q:
        growth = {}
        for comp in companies:
            g = df[df["Company"].str.lower() == comp].sort_values("Fiscal Year")
            if len(g) > 1:
                latest = g.iloc[-1]["Total Revenue (USD B)"]
                prev = g.iloc[-2]["Total Revenue (USD B)"]
                growth[comp] = (latest - prev) / prev * 100
        top_company = max(growth, key=growth.get)
        return f"{top_company.title()} had the highest recent revenue growth ({growth[top_company]:.2f}%)."
    
    if "cash flow" in q:
        if not match_company:
            return "Please specify a company."
        info = canned[match_company]
        return f"{match_company.title()} cash flow from operations ({info['latest_year']}): {info['cash_flow']} USD B."
    
    return "Sorry, I can only answer predefined queries."

# Example test
answer_query("What is the total revenue for Apple?")


'Apple total revenue (2023): 383.29 USD B.'

In [5]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ("exit", "quit", "next"):
        print("Bot: Goodbye! Moving to next cell...")
        break
    print("Bot:", answer_query(user_input))


You:  hi


Bot: Sorry, I can only answer predefined queries.


You:  next


Bot: Goodbye! Moving to next cell...


In [6]:
readme_content = f"""
# Financial Chatbot Prototype

## Description
A minimal chatbot that answers predefined financial queries for Apple, Microsoft, and Tesla using data from `financial_data.csv`.

## Supported Queries
- What is the total revenue for <company>?
- How has net income changed over the last year for <company>?
- Which company had the highest recent revenue growth?
- What is the cash flow from operating activities for <company>?

## How to Run in Jupyter Notebook
1. Place `financial_data.csv` in the same folder as this notebook.
2. Open the notebook in Jupyter.
3. Run all cells.
4. Use the CLI cell to type questions, or run the optional Flask app cell for a browser interface.

## Data Source
Data comes from `financial_data.csv` with the following columns:
- Company
- Fiscal Year
- Total Revenue (USD B)
- Net Income (USD B)
- Total Assets (USD B)
- Total Liabilities (USD B)
- Cash Flow from Operating Activities (USD B)

## Limitations
- Only answers 4 types of predefined queries.
- Company names must be present in the dataset.
- No live data fetching — answers are static from the CSV.
"""

with open("README.md", "w") as f:
    f.write(readme_content)

print("README.md created.")


README.md created.
